Análise das diferenças de salários entre homens e mulheres no campo de data science no Brasil. A análise tem componentes gráficos e estatístico:
* [Presença em data science (Homens vs Mulheres)](#presenca)
* [Escolaridade e Tipo de emprego para homens e Mulheres: Análise porcentual](#escol)
* [Questão salarial, homens ganham mais do que as mulheres?](#salario)
* [Significância estatística das diferenças](#significancia)
* [Conclusões](#conclusao)

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import missingno as msno
import matplotlib.pyplot as plt
import scipy.stats as st
import matplotlib.patches as mpatches

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_ds = pd.read_csv('/kaggle/input/pesquisa-data-hackers-2019/datahackers-survey-2019-anonymous-responses.csv')
print(df_ds.shape)
df_ds.head()

# Presênça em data science (Homens vs Mulheres)
<a id='presenca'></a>

In [ ]:
sns.countplot(df_ds["('P2', 'gender')"])
plt.show()

## Escolaridade e Tipo de emprego para homens e Mulheres: Análise porcentual
<a id='escol'></a>

In [ ]:
col_search = "('P8', 'degreee_level')"

df_agg = df_ds.groupby([col_search,"('P2', 'gender')"]).agg({col_search:'count',})
df_agg = df_agg/df_agg.groupby(level=(1)).sum()

df_agg.reset_index(level=1, inplace=True)

plt.figure(figsize=(16,6))

colors = ['orange' if s == 'Feminino' else 'lightblue' for s in df_agg["('P2', 'gender')"]]
plt.bar(np.arange(len(df_agg.index)),df_agg[col_search],color=colors)
plt.xticks(np.arange(len(df_agg.index)),df_agg.index,rotation=90)

red_patch = mpatches.Patch(color='orange', label='Feminino')
blue_patch = mpatches.Patch(color='lightblue', label='Masculino')

plt.legend(handles=[red_patch, blue_patch])
plt.show()

In [ ]:
df_ds["('P10', 'job_situation')"].value_counts()*100/df_ds.size

In [ ]:
col_search = "('P10', 'job_situation')"

df_agg = df_ds.groupby([col_search,"('P2', 'gender')"]).agg({col_search:'count',})
df_agg = df_agg/df_agg.groupby(level=(1)).sum()

df_agg.reset_index(level=1, inplace=True)

plt.figure(figsize=(16,6))

colors = ['orange' if s == 'Feminino' else 'lightblue' for s in df_agg["('P2', 'gender')"]]
plt.bar(np.arange(len(df_agg.index)),df_agg[col_search],color=colors)
plt.xticks(np.arange(len(df_agg.index)),df_agg.index,rotation=90)

red_patch = mpatches.Patch(color='orange', label='Feminino')
blue_patch = mpatches.Patch(color='lightblue', label='Masculino')

plt.legend(handles=[red_patch, blue_patch])
plt.show()

# Questão salarial, homens ganham mais do que as mulheres?
<a id='salario'></a>

Devido a que o salário está dado por faixa, iremos aproximar o valor de cada faixa usando a média respectiva.

In [ ]:
def transform_string(st):
    
    limits = [float(s[:-4].replace('.','')) for s in st.split() if s[-4:]=='/mês']
    if len(limits) == 2:
        value = np.mean(np.array(limits))
    elif limits[0]>10000:
        value = limits[0]
    else:
        value = limits[0]/2
    return value

df_ds["('P16', 'salary_range')"].fillna('0.0/mês',inplace=True)

df_ds['salary'] = df_ds.apply(lambda row: transform_string(row["('P16', 'salary_range')"]),axis=1)

df_ds[["('P16', 'salary_range')",'salary']].head()

In [ ]:
sns.distplot(df_ds["salary"],bins=20)
plt.show()

In [ ]:
sns.distplot(df_ds.loc[df_ds["('P2', 'gender')"]=='Masculino',["salary"]],bins=20,label='Masculino')
sns.distplot(df_ds.loc[df_ds["('P2', 'gender')"]=='Feminino',["salary"]],bins=20,label='Feminino')
plt.legend(loc=0)
plt.show()

In [ ]:
col_search = "('P10', 'job_situation')"

medianprops = dict(linestyle='-', linewidth=4)
bp_dict = df_ds.boxplot(column='salary',by=[col_search,"('P2', 'gender')"],rot=90,
                        figsize=(20,8),fontsize=16,grid=True,return_type='both',patch_artist = True,
                        medianprops=medianprops)

df_agg = df_ds.groupby([col_search,"('P2', 'gender')"]).agg({col_search:'count',})
df_agg = df_agg/df_agg.groupby(level=(1)).sum()

df_agg.reset_index(level=1, inplace=True)

colors = ['orange' if s == 'Feminino' else 'lightblue' for s in df_agg["('P2', 'gender')"]]

for row_key, (ax,row) in bp_dict.iteritems():
    ax.set_xlabel('')
    for i,box in enumerate(row['boxes']):
        box.set_facecolor(colors[i])

red_patch = mpatches.Patch(color='orange', label='Feminino')
blue_patch = mpatches.Patch(color='lightblue', label='Masculino')

plt.legend(handles=[red_patch, blue_patch])
plt.title('Diferenças de salário por tipo de trabalho entre homens e mulheres')
plt.show()

Os gráficos mostram que mulheres ganham, na média, menos do que os homems em todas as vagas do mercado. Mulheres também tem uma ligeira preferência pela academia. É sabido que salários na pesquisa e academia são menores quando comparados com o mercado de trabalho, e isto pode estar relacionado com os dois pontos anteriores. Infelizmente, não temos dados de salário na parte da área acadêmica e de pesquisa para aprofundar sobre isto. Para verificar a significancia estatística destas diferenças faremos a análisa por categoria de trabalho usando estatística inferencial.

# Significância estatística das diferenças
<a id = 'significancia'></a>

In [ ]:
df_stats = df_ds.groupby("('P2', 'gender')").agg({'salary':['mean','std','count']})
mean_fem, std_fem, n_fem = df_stats.loc['Feminino']
mean_mas, std_mas, n_mas = df_stats.loc['Masculino']

media_dif = mean_mas - mean_fem

beta = 0.975 #Bicaudal
z_norm = st.norm.ppf(beta)

std_dif = np.sqrt((std_mas**2)/n_mas+(std_fem**2)/n_fem)
d = z_norm*std_dif

z = media_dif/std_dif
p_value = 1 - st.norm.cdf(z)

df_st = pd.DataFrame([['global',n_fem,n_mas,media_dif,std_dif,(media_dif-d),(media_dif+d),z,p_value],],
                     columns=['job','n feminino','n masculino','mean_dif','std_dif','inter_inf','inter_sup','z','p_value'])
df_st

In [ ]:
col_search = "('P10', 'job_situation')"

for job_type in ['Empregado (CTL)', 'Estagiário', 'Freelancer','Empreendedor ou Empregado (CNPJ)',
                 'Servidor Público',]:


    df_stats = df_ds.loc[df_ds[col_search]==job_type].groupby("('P2', 'gender')").agg({'salary':['mean','std','count']})
    mean_fem, std_fem, n_fem = df_stats.loc['Feminino']
    mean_mas, std_mas, n_mas = df_stats.loc['Masculino']

    media_dif = mean_mas - mean_fem

    std_dif = np.sqrt((std_mas**2)/n_mas+(std_fem**2)/n_fem)
    d = z_norm*std_dif

    z = media_dif/std_dif
    p_value = 1 - st.norm.cdf(z)

    df_st = df_st.append(pd.DataFrame([[job_type,n_fem,n_mas,media_dif,std_dif,(media_dif-d),(media_dif+d),z,p_value],],
                         columns=['job','n feminino','n masculino','mean_dif','std_dif','inter_inf','inter_sup','z','p_value']),
                         ignore_index=True)
df_st

# Discriminação por tempo de experiência em data science


In [ ]:
col_search = "('P17', 'time_experience_data_science')"

df_agg = df_ds.groupby([col_search,"('P2', 'gender')"]).agg({col_search:'count',})
df_agg = df_agg/df_agg.groupby(level=(1)).sum()

df_agg.reset_index(level=1, inplace=True)

plt.figure(figsize=(16,6))

colors = ['orange' if s == 'Feminino' else 'lightblue' for s in df_agg["('P2', 'gender')"]]
plt.bar(np.arange(len(df_agg.index)),df_agg[col_search],color=colors)
plt.xticks(np.arange(len(df_agg.index)),df_agg.index,rotation=90)

red_patch = mpatches.Patch(color='orange', label='Feminino')
blue_patch = mpatches.Patch(color='lightblue', label='Masculino')

plt.legend(handles=[red_patch, blue_patch])
plt.show()

In [ ]:
medianprops = dict(linestyle='-', linewidth=4)
bp_dict = df_ds.boxplot(column='salary',by=[col_search,"('P2', 'gender')"],rot=90,
                        figsize=(20,8),fontsize=16,grid=True,return_type='both',patch_artist = True,
                        medianprops=medianprops)

df_agg = df_ds.groupby([col_search,"('P2', 'gender')"]).agg({col_search:'count',})
df_agg = df_agg/df_agg.groupby(level=(1)).sum()

df_agg.reset_index(level=1, inplace=True)

colors = ['orange' if s == 'Feminino' else 'lightblue' for s in df_agg["('P2', 'gender')"]]

for row_key, (ax,row) in bp_dict.iteritems():
    ax.set_xlabel('')
    for i,box in enumerate(row['boxes']):
        box.set_facecolor(colors[i])

red_patch = mpatches.Patch(color='orange', label='Feminino')
blue_patch = mpatches.Patch(color='lightblue', label='Masculino')

plt.legend(handles=[red_patch, blue_patch])
plt.title('Diferenças de salário por tipo de trabalho entre homens e mulheres')
plt.show()

In [ ]:
df_st = pd.DataFrame([], columns=['job','n feminino','n masculino','mean_dif','std_dif','inter_inf','inter_sup','z','p_value'])

for exp_type in ['Não tenho experiência na área de dados','Menos de 1 ano','de 1 a 2 anos','de 2 a 3 anos','de 4 a 5 anos','de 6 a 10 anos','Mais de 10 anos',]:


    df_stats = df_ds.loc[df_ds[col_search]==exp_type].groupby("('P2', 'gender')").agg({'salary':['mean','std','count']})
    mean_fem, std_fem, n_fem = df_stats.loc['Feminino']
    mean_mas, std_mas, n_mas = df_stats.loc['Masculino']

    media_dif = mean_mas - mean_fem

    std_dif = np.sqrt((std_mas**2)/n_mas+(std_fem**2)/n_fem)
    d = z_norm*std_dif

    z = media_dif/std_dif
    p_value = 1 - st.norm.cdf(z)

    df_st = df_st.append(pd.DataFrame([[exp_type,n_fem,n_mas,media_dif,std_dif,(media_dif-d),(media_dif+d),z,p_value],],
                         columns=['job','n feminino','n masculino','mean_dif','std_dif','inter_inf','inter_sup','z','p_value']),
                         ignore_index=True)
df_st

# Conclusão
<a id='conclusao'></a>

* Depois de verificar a significância estatística das diferenças de salário para mulheres e homens por tipo de trabalho, podemos concluir que na média existe sim uma diferença de salário no mercado, sendo que mulheres ganham aproximadamente 1200 reais a menos do que os homens para vagas CLT.
* Essa diferença se extende quando olhamos para as faixas salariais. O teste estatístico mostra que, tirando tempo de experiência de menos de 1 ano e entre 6 a 10 anos onde a diferença é pequena, os homens ganham mais do que as mulheres.
* Ainda que mulheres tenham maior preferência por trabalhar na parte acadêmica e de pesquisa, em comparação com os homens, não temos como explorar questão de salários nesta área.
* Fica a questão do porque da baixa presença de mulheres em data science...

# Limitações


Ainda que se apresentou diferença nas outras áreas do mercado (diferente a CLT) esta não é conclusiva dado o baixo número de amostras por categoria e a baixa sgnificância estatística. Também é importante notar que o salário usado não é o valor real, mas sim uma aproximação dada a partir da faixa.